In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
FOLDER_PATH = 'data/최종/'

In [3]:
class company_prerocessing():
#     def __init__()
    def open_df(folder_path):
        df = pd.read_csv(folder_path)

        return df
    
    def cis_preprocessing(df_cis, cmp_list):
        
        if '01-20' in df_cis.columns[6]:
            df_cis_1 = df_cis.iloc[:, [1, 6, 7]]
        else:
            df_cis_1 = df_cis.iloc[:, [1, 7, 8]]
                
        df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2]]
        df_cis_1 = df_cis_1.T
        
        
        df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
        df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
        df_cis_1.drop('label_ko', inplace=True, axis =1)
        
        df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})

        df_cis_1['company'] = cmp_list
        df_cis_1['date'] = df_cis_1['date'].str[9:]
        print('cis_preprocessing complete')

        # df_cis_1.to_csv('data/최종/포괄손익계산서 전처리/'+cmp_list+'.csv', index=False)

        return df_cis_1
    
        
    def bs_preprocessing(df_bs, cmp_list):
        try:
            if df_bs.columns[6].isdigit() == True:
                df_bs_1 = df_bs.iloc[:, [1, 6, 7, 8]]
            elif df_bs.columns[7].isdigit() == True:
                df_bs_1 = df_bs.iloc[:, [1, 7, 8, 9]]
            else:
                df_bs_1 = df_bs.iloc[:, [1, 8, 9, 10]]
        except:
            df_bs_1 = df_bs.iloc[:, 0:4]
            
        df_bs_1.columns = ['', df_bs_1.columns[1], df_bs_1.columns[2], df_bs_1.columns[3]]
        df_bs_1 = df_bs_1.T
        
        df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
        df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
        df_bs_1.drop('label_ko', inplace=True, axis =1)
            
        df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})
        
       
        df_bs_1['company'] = cmp_list
        print('bs_preprocessing complete')
        
        # df_bs_1.to_csv('data/최종/재무재표 전처리/'+cmp_list+'.csv', index=False)
        return df_bs_1

    def save_df(folder_path, df, cmp_list):
        df.to_csv(folder_path + cmp_list + '.csv', index=False)

    def total_dataframe(self, df_cis_1, df_bs_1, df_cf_1, cmp_list):
        data_total = pd.merge(df_cis_1, df_bs_1, on='date')
        data_total = pd.merge(data_total, df_cf_1, on='date')
        data_total.to_csv('data/total/' + cmp_list + ' total.csv', index=False)

        return data_total

In [23]:
# if '__ma|in__' == __name__:
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 13, '매출채권': 13, '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}

bs_a = pd.DataFrame(columns=['date', 'company'])
cis_a = pd.DataFrame(columns=['date', 'company'])

file_list1 = os.listdir(FOLDER_PATH +'재무상태표/')
file_list2 = os.listdir(FOLDER_PATH +'포괄손익계산서/')



for cmp_list1, cmp_list2 in zip(file_list1, file_list2):

    # 재무 상태표
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '재무상태표/' + cmp_list1))
    df_bs_1 = company_prerocessing.bs_preprocessing(df, cmp_list1[:-13])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + '재무재표 전처리/'), df_bs_1, cmp_list1[:-13])

    list1 = ['date', 'company']
    for var1, var1_ in var_1_dic.items():
        for bs in df_bs_1.columns:
            if var1 in bs:
                if var1_ >= len(bs):
                    list1.append(bs)
    print(df_bs_1[list1])
    bs_a = pd.concat([bs_a, df_bs_1[list1]])




    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '포괄손익계산서/' + cmp_list2))
    df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + '포괄손익계산서 전처리/'), df_cis_1, cmp_list2[:-15])



    list2 = ['date', 'company']
    for var2, var2_ in var_2_dic.items():
        for cis in df_cis_1.columns:
            if var2 in cis:
                if var2_ >= len(cis):
                    list1.append(cis)
    print(df_cis_1[list2])
    cis_a = pd.concat([cis_a, df_cis_1[list2]])



bs_a.reset_index(inplace=True, drop=True)
cis_a.reset_index(inplace=True, drop=True)
company_prerocessing.save_df(os.path.join(FOLDER_PATH + '재무재표 전처리/'), bs_a, '재무상태표 통합')
company_prerocessing.save_df(os.path.join(FOLDER_PATH + '포괄손익계산서 전처리/'), cis_a, '포괄손익계산서 통합')


bs_preprocessing complete
       date company           자본총계            자산총계  매입채무 및 기타유동채무  \
0  20111231    금강제강  61279687814.0  194206670604.0  39110749325.0   
1  20101231    금강제강  72211501426.0  179481870466.0  41074511130.0   
2  20091231    금강제강  55248061746.0  120141443625.0  21023313155.0   

   매출채권 및 기타유동채권            부채총계            유동부채            유동자산  \
0  49725295614.0  132926982790.0  113985044750.0  100969321003.0   
1  61178350458.0  107270369040.0   85302556870.0   99806911954.0   
2  37786731906.0   64893381879.0   50998611676.0   57435150864.0   

            재고자산  
0  37436698391.0  
1  33343253080.0  
2  14166765495.0  
cis_preprocessing complete
       date company
0  20111231    금강제강
1  20101231    금강제강
bs_preprocessing complete
       date company           자본총계            자산총계  매입채무 및 기타유동채무  \
0  20111231    배명금속  95296464290.0  208619609146.0  26367131866.0   
1  20101231    배명금속  77745475250.0  179399062436.0  18536080027.0   
2  20091231    배명금속  8261021

In [24]:
cis_a

,date,company
0,20111231,금강제강
1,20101231,금강제강
2,20111231,배명금속
3,20101231,배명금속
4,20111231,풍림산업
5,20101231,풍림산업


In [25]:
bs_a

,date,company,자본총계,자산총계,매입채무 및 기타유동채무,매출채권 및 기타유동채권,부채총계,유동부채,유동자산,재고자산
0,20111231,금강제강,61279687814.0,194206670604.0,39110749325.0,49725295614.0,132926982790.0,113985044750.0,100969321003.0,37436698391.0
1,20101231,금강제강,72211501426.0,179481870466.0,41074511130.0,61178350458.0,107270369040.0,85302556870.0,99806911954.0,33343253080.0
2,20091231,금강제강,55248061746.0,120141443625.0,21023313155.0,37786731906.0,64893381879.0,50998611676.0,57435150864.0,14166765495.0
3,20111231,배명금속,95296464290.0,208619609146.0,26367131866.0,30166438816.0,113323144856.0,106984768096.0,120559651206.0,66618482419.0
4,20101231,배명금속,77745475250.0,179399062436.0,18536080027.0,26118974181.0,101653587186.0,95049265962.0,94315952312.0,55218399050.0
5,20091231,배명금속,82610217591.0,171171430848.0,14123945120.0,14175850742.0,88561213257.0,82158201786.0,89565588511.0,56707866704.0
6,20111231,풍림산업,152837431616.0,1374255960342.0,516846395394.0,759265274862.0,1221418528726.0,752955724323.0,919718310569.0,30002292372.0
7,20101231,풍림산업,185149189773.0,1188801152279.0,371141947782.0,658773549500.0,1003651962506.0,954388334556.0,771447482160.0,34818379461.0
8,20091231,풍림산업,261243875520.0,1269465881167.0,327249400675.0,713340312689.0,1008222005647.0,506509390912.0,875186217415.0,35314961520.0


In [13]:
pd.merge(cis_a, bs_a, on=['date','company'])

,date,company,매출총이익,법인세비용차감전순이익(손실),당기순이익(손실),자본총계,자산총계,부채총계,유동부채,유동자산,재고자산
0,20111231,금강제강,591907148.0,-13509435372.0,-11260872179.0,61279687814.0,194206670604.0,132926982790.0,113985044750.0,100969321003.0,37436698391.0
1,20101231,금강제강,13444476213.0,2073005833.0,1519307289.0,72211501426.0,179481870466.0,107270369040.0,85302556870.0,99806911954.0,33343253080.0
2,20111231,배명금속,8510337046.0,376240588.0,627781200.0,95296464290.0,208619609146.0,113323144856.0,106984768096.0,120559651206.0,66618482419.0
3,20101231,배명금속,7700935418.0,-5482424803.0,-5245282081.0,77745475250.0,179399062436.0,101653587186.0,95049265962.0,94315952312.0,55218399050.0
4,20111231,풍림산업,2072700894.0,-41366351473.0,-35852563764.0,152837431616.0,1374255960342.0,1221418528726.0,752955724323.0,919718310569.0,30002292372.0
5,20101231,풍림산업,29044164067.0,-91069640922.0,-74624076085.0,185149189773.0,1188801152279.0,1003651962506.0,954388334556.0,771447482160.0,34818379461.0


In [55]:
df_cis_1
df_cis_1

,date,label_ko,수익,국내공사수입,해외공사수입,임대수입,제품매출수입,기타매출수입,분양매출수입,매출원가,...,기타포괄손익,해외사업환산손익,매도가능금융자산평가손익,보험수리적손익,총포괄손익,"포괄손익, 지배기업의 소유주에게 귀속되는 지분","포괄손익, 비지배지분",기본주당이익(손실),희석주당이익(손실),company
0,0,20110101-20111231,수익,국내공사수입,해외공사수입,임대수입,제품매출수입,기타매출수입,분양매출수입,매출원가,...,기타포괄손익,해외사업환산손익,매도가능금융자산평가손익,보험수리적손익,총포괄손익,"포괄손익, 지배기업의 소유주에게 귀속되는 지분","포괄손익, 비지배지분",기본주당이익(손실),희석주당이익(손실),풍림산업
1,1,20100101-20101231,1040133868537.0,998594426801.0,5139959955.0,5044777566.0,16346058014.0,14988123813.0,20522388.0,1038061167643.0,...,3540805607.0,2240108630.0,1786556576.0,-485859599.0,-32311758157.0,-32311758157.0,NaN,-2021.0,-2021.0,풍림산업
2,2,"('연결재무제표',)",1115226030432.0,1064823068372.0,10889326571.0,4976794251.0,23829611248.0,10707229990.0,0.0,1086181866365.0,...,-1470609662.0,-628693573.0,-844092501.0,2176412.0,-76094685747.0,-76094685747.0,NaN,-4207.0,-4207.0,풍림산업


In [12]:
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 13, '매출채권': 13, '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}
var_2_dic = {'date':4, '매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10,
         'company': 7}

cis_a = pd.DataFrame(0, index= np.arange(15), columns=['date', '매출총이익', '법인세비용차감전순이익', '당기순이익'])

file_list1 = os.listdir(FOLDER_PATH +'재무상태표/')
file_list2 = os.listdir(FOLDER_PATH +'포괄손익계산서/')

count = 0

In [13]:
file_list2

['금강제강 포괄손익계산서 원본.csv', '배명금속 포괄손익계산서 원본.csv', '풍림산업 포괄손익계산서 원본.csv']

In [14]:
cis_a

,date,매출총이익,법인세비용차감전순이익,당기순이익
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [15]:
df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '포괄손익계산서/' + file_list2[0]))
df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])

for var2, var2_ in var_2_dic.items():
    for cis in df_cis_1.columns:
        if var2 in cis:
            if var2_ >= len(cis):
                print(cis)
                cis_a.iloc[0][count] = df_cis_1[cis]
                count += 1

NameError: name 'cmp_list2' is not defined

In [16]:
for cmp_list2 in file_list2:
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '포괄손익계산서/' + cmp_list2))
    df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])
        
    for var2, var2_ in var_2_dic.items():
        for cis in df_cis_1.columns:
            if var2 in cis:
                if var2_ >= len(cis):
                    print(cis)
                    cis_a.iloc[:, count] = df_cis_1[cis]
                    count += 1

KeyError: "['level_0'] not found in axis"

In [56]:
os.path.join(FOLDER_PATH , file_list[0])

NameError: name 'file_list' is not defined

In [ ]:
file_list[0]

In [119]:
df_cis = company_prerocessing.open_df(os.path.join(FOLDER_PATH +'포괄손익계산서/'+file_list2[1]))

In [120]:
df_cis.head()

,"[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW)","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).1","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).2","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).3","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).4","[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW).5",20110101-20111231,20100101-20101231
0,concept_id,label_ko,label_en,class0,class1,class2,"('연결재무제표',)","('연결재무제표',)"
1,ifrs_Revenue,수익,Revenue(Sales),포괄손익계산서 [abstract],수익,NaN,81567060964.0,102325981917.0
2,ifrs_CostOfSales,매출원가,Cost of sales,포괄손익계산서 [abstract],매출원가,NaN,73056723918.0,94625046499.0
3,ifrs_GrossProfit,매출총이익,Gross profit,포괄손익계산서 [abstract],매출총이익,NaN,8510337046.0,7700935418.0
4,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,Selling general administrative expenses ...,포괄손익계산서 [abstract],판매비와관리비,NaN,5065406425.0,9233609836.0


In [135]:
if '01-20' in df_cis.columns[6]:
    df_cis_1 = df_cis.iloc[:, [1, 6, 7]]
else:
    df_cis_1 = df_cis.iloc[:, [1, 7, 8]]


df_cis_1.columns

df_cis_1.columns[1]

df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2]]
df_cis_1

df_cis_1 = df_cis_1.T

df_cis_1

df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])

df_cis_1

df_cis_1 = df_cis_1.drop(df_cis_1.index[0])

df_cis_1

df_cis_1.drop('label_ko', inplace=True, axis =1)

df_cis_1

df_cis_1

,수익,매출원가,매출총이익,판매비와관리비,영업이익(손실),기타수익,기타비용,금융수익,금융원가,법인세비용차감전순이익(손실),...,당기순이익(손실),지배기업의 소유주에게 귀속되는 당기순이익(손실),비지배지분에 귀속되는 당기순이익(손실),연결기타포괄손익,해외사업환산손익,보험수리적손익,연결당기총포괄이익,"포괄손익, 지배기업의 소유주에게 귀속되는 지분","포괄손익, 비지배지분",기본주당이익(손실)
20110101-20111231,81567060964.0,73056723918.0,8510337046.0,5065406425.0,3444930621.0,3330603897.0,2058192368.0,198325041.0,4539426603.0,376240588.0,...,627781200.0,627781200.0,0.0,1720221602.0,1769414358.0,-49192756.0,2348002802.0,2348002802.0,0.0,7.0
20100101-20101231,102325981917.0,94625046499.0,7700935418.0,9233609836.0,-1532674418.0,1857924763.0,1751015242.0,100126194.0,4156786100.0,-5482424803.0,...,-5245282081.0,-5245282081.0,0.0,378372976.0,456954301.0,-78581325.0,-4866909105.0,-4866909105.0,0.0,-79.0


In [ ]:
        df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
        # df_cis_1['date'] = df_cis_1['date'].str[9:]
#         df_cis_1.drop(['level_0', 'label_ko'], axis=1, inplace=True)

        df_cis_1['company'] = cmp_list
        print('cis_preprocessing complete')

In [ ]:
'01-20' in df_cis.columns[6]

In [ ]:
df_cis.columns[6].isdigit() == True

In [ ]:
df_cis.head()

In [ ]:
df_cis_1 = df_cis.iloc[:, [1, 7,8]]

In [ ]:
df_cis_1.head()

In [ ]:
df_cis_1.columns = ['', df_cis_1.columns[1][0], df_cis_1.columns[2][0]]

In [ ]:
df_cis_1 = df_cis_1.T

In [ ]:
df_cis_1.columns = df_cis_1.columns.astype('object')

In [ ]:
for i in range(len(df_cis_1.columns)):
            df_cis_1.columns.values[i] = df_cis_1.iloc[0, :][i]

In [ ]:
df_cis_1.reset_index(inplace=True)
df_cis_1.drop(0, inplace=True)

In [ ]:
for i in range(2):
    df_cis_1.iloc[i, 0] = df_cis.columns.values[i+6]

In [ ]:
df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
df_cis_1.drop(['level_0', 'label_ko'], axis=1, inplace=True)

In [ ]:
def cis_preprocessing(df_cis):
    df_cis_1 = df_cis.iloc[:, [1, 7, 8, 9]]
    df_cis_1.columns = ['', df_cis_1.columns[1][0], df_cis_1.columns[2][0], df_cis_1.columns[3][0]]
    df_cis_1 = df_cis_1.T
    df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
    df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
    df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
    df_cis_1['date'] = df_cis_1['date'].str[9:]
    print('cis_proprocessing complete')

    return df_cis_1

In [ ]:
file_list[4]

In [ ]:
df = company_prerocessing.open_df(os.path.join(FOLDER_PATH , file_list[8]))
print(file_list[8])
df.head()

In [ ]:
df_bs_1 = df.iloc[:, 0:4]

In [ ]:
df_bs_1.head()

In [ ]:
df_bs_1.columns = ['', df_bs_1.columns[1][0], df_bs_1.columns[2][0], df_bs_1.columns[3][0]]

In [ ]:
df_bs_1

In [ ]:
df_bs_1 = df_bs_1.T

In [ ]:
df_bs_1.head()

In [ ]:
df_bs_1.columns = df_bs_1.columns.astype('object')

In [ ]:
for i in range(len(df_bs_1.columns)):
    
    df_bs_1.columns.values[i] = df_bs_1.iloc[0, :][i]

In [ ]:
df_bs_1.reset_index(inplace=True)

In [ ]:
df_bs_1

In [ ]:
df_bs_1.drop(0, inplace=True)

In [ ]:
df_bs_1.head()

In [ ]:
df_bs_1.iloc[1, 0]

In [ ]:
df.columns.values[1]

In [ ]:
df_bs_1.iloc[0, 0] = df.columns.values[1]
df_bs_1.iloc[1, 0] = df.columns.values[2]
df_bs_1.iloc[2, 0] = df.columns.values[3]

In [ ]:
df_bs_1

In [ ]:
# df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
# df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})

In [ ]:
df_bs_1.drop(['level_0', 'label_ko'], axis=1, inplace=True)

In [21]:
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 13, '매출채권': 13, '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}

In [17]:
df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '재무상태표/' + file_list1[0]))
df_bs_1 = company_prerocessing.bs_preprocessing(df, cmp_list1[:-13])
company_prerocessing.save_df(os.path.join(FOLDER_PATH + '재무재표 전처리/'), df_bs_1, cmp_list1[:-13])

bs_preprocessing complete


In [29]:
list1 = ['date', 'company']
for var1, var1_ in var_1_dic.items():
    for bs in df_bs_1.columns:
        if var1 in bs:
            if var1_ == len(bs):
                print(bs)
                list1.append(bs)
# print(df_bs_1[list1])
# bs_a = pd.concat([bs_a, df_bs_1[list1]])

자본총계
자산총계
매입채무 및 기타유동채무
매출채권 및 기타유동채권
부채총계
유동부채
유동자산
재고자산


In [20]:
len('매출채권 및 기타유동채권')

13